In [1]:
import gymnasium as gym
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = ""
import jax
import jax.numpy as jnp

import numpy as np

# from irbfn_mpc.explicit_planner import ExplicitPlanner
import irbfn_mpc.explicit_planner as ie
import irbfn_mpc.nonlinear_dmpc as nd

2024-09-09 19:51:58.488047: W external/xla/xla/service/gpu/nvptx_compiler.cc:760] The NVIDIA driver's CUDA version is 12.4 which is older than the ptxas CUDA version (12.5.40). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [2]:
from IPython.display import HTML, display
%matplotlib inline

In [3]:
env = gym.make(
    "f1tenth_gym:f1tenth-v0",
    config={
        "map": "Spielberg_blank",
        "observation_config": {
            "type": "features",
            "features": [
                "pose_x",
                "pose_y",
                "delta",
                "linear_vel_x",
                "linear_vel_y",
                "pose_theta",
                "ang_vel_z",
                "beta",
            ],
        },
        "num_agents": 1,
        "control_input": "accl",
    },
    render_mode="rgb_array",
)
env = gym.wrappers.RecordVideo(env, "video_explicit_nmpc_frenet")

/usr/local/lib/python3.12/dist-packages/f1tenth_gym/envs/action.py:155: UserWarning: Only one control mode specified, using accl for longitudinal control and defaulting to steering speed for steering
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /home/irbfn/scripts/video_explicit_nmpc_frenet folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


In [4]:
track = env.unwrapped.track

In [5]:
import chex
chex.clear_trace_counter()

In [11]:
from importlib import reload
reload(ie)

<module 'irbfn_mpc.explicit_planner' from '/home/irbfn/src/irbfn_mpc/explicit_planner.py'>

In [12]:
planner = ie.ExplicitFrenetPlanner(track=track)

In [13]:
env.unwrapped.add_render_callback(planner.render_waypoints)
env.unwrapped.add_render_callback(planner.render_local_plan)
env.unwrapped.add_render_callback(planner.render_mpc_sol)

In [18]:
poses = np.array(
    [
        [
            env.unwrapped.track.raceline.xs[0],
            env.unwrapped.track.raceline.ys[0],
            env.unwrapped.track.raceline.yaws[0],
        ]
    ]
)
obs, info = env.reset(options={"poses": poses})
done = False

step = 0

while not done and step <= 4000:
    if step <= 100:
        accl = 10.0
        steerv = 0.0
    current_state = obs["agent_0"]
    accl, steerv = planner.plan(current_state)
    # accl, steerv = planner2.plan(current_state)
    action = env.action_space.sample()
    action[0] = [steerv, accl]
    # action[0] = [steerv[0], accl[0]]
    obs, step_reward, done, truncated, info = env.step(action)
    step += 1
    # print(current_state)
    # print(f"accl {accl} steerv {steerv}")

env.close()

invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid output
invlaid ou

Moviepy - Done !
Moviepy - video ready /home/irbfn/scripts/video_explicit_nmpc_frenet/rl-video-episode-0.mp4


In [19]:
import glob
import io
import base64

for video_file in glob.glob("video_explicit_nmpc_frenet/*.mp4"):
    video = io.open(video_file, "rb").read()
    encoded = base64.b64encode(video).decode("ascii")
    display(
        HTML(
            f"""<video width="800" height="auto" controls>
                <source src="data:video/mp4;base64,{encoded}" type="video/mp4" />
            </video>"""
        )
    )